In [168]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from typing import Counter
import pandas as pd
import numpy as np

In [169]:
file_path = './dataset/PCOS_data.csv'
data = pd.read_csv(file_path)

# data.info(), data.head(), data.describe()

In [170]:
data_cleaned = data.replace(',', '.', regex=True)

# convertendo colunas para os tipos numéricos apropriados
for col in data_cleaned.columns:
  try:
    data_cleaned[col] = pd.to_numeric(data_cleaned[col])
  except ValueError:
    pass

# verificando e remover duplicatas
duplicates = data_cleaned.duplicated().sum()
data_cleaned = data_cleaned.drop_duplicates()

# verificando valores de cada classe para balanceamento
class_distribution = data_cleaned['PCOS (Y/N)'].value_counts()

duplicates, class_distribution

(0,
 PCOS (Y/N)
 0    364
 1    177
 Name: count, dtype: int64)

In [171]:
# separando características (X) e rótulos (y)
X = data_cleaned.drop(columns=['PCOS (Y/N)'])
y = data_cleaned['PCOS (Y/N)']

# balanceamento dos dados com SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

print(f"Distribuição das classes após SMOTE: {Counter(y_balanced)}")

# dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

# normalizando os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# testando diferentes valores de k para o KNN
k_values = range(1, 21)
accuracies = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)
    accuracies.append(accuracy_score(y_test, y_pred))

best_k = k_values[accuracies.index(max(accuracies))]
print(f"Melhor valor de k: {best_k}")

Distribuição das classes após SMOTE: Counter({0: 364, 1: 364})
Melhor valor de k: 1


In [172]:
# treinando o modelo com o melhor valor de k
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(X_train_scaled, y_train)

# avaliação do modelo
y_pred_best = knn_best.predict(X_test_scaled)
conf_matrix = confusion_matrix(y_test, y_pred_best)
class_report = classification_report(y_test, y_pred_best)

cv_scores_all_features = cross_val_score(knn_best, X_train_scaled, y_train, cv=5, scoring='accuracy')
mean_accuracy_all = np.mean(cv_scores_all_features)
std_accuracy_all = np.std(cv_scores_all_features)

print("Relatório de Classificação:\n", class_report)
print("Matriz de Confusão:\n", conf_matrix)
print(f"\nValidação cruzada com todas as características:\nAcurácia média: {mean_accuracy_all:.4f}, Desvio padrão: {std_accuracy_all:.4f}")

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.96      0.83      0.89       110
           1       0.85      0.96      0.90       109

    accuracy                           0.89       219
   macro avg       0.90      0.90      0.89       219
weighted avg       0.90      0.89      0.89       219

Matriz de Confusão:
 [[ 91  19]
 [  4 105]]

Validação cruzada com todas as características:
Acurácia média: 0.8408, Desvio padrão: 0.0412


### Treinando com as K Melhores

In [173]:
# seleção de características com SelectKBest
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]

print("Características selecionadas:", selected_features)

Características selecionadas: Index(['Weight (Kg)', 'Cycle(R/I)', 'AMH(ng/mL)', 'Weight gain(Y/N)',
       'hair growth(Y/N)', 'Skin darkening (Y/N)', 'Pimples(Y/N)',
       'Fast food (Y/N)', 'Follicle No. (L)', 'Follicle No. (R)'],
      dtype='object')


In [174]:
# balanceamento dos dados com SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_new, y)

print(f"Distribuição das classes após SMOTE: {Counter(y_balanced)}")

# divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

# normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Distribuição das classes após SMOTE: Counter({0: 364, 1: 364})


In [175]:
# treinamento e avaliação do modelo com o melhor k
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(X_train_scaled, y_train)

cv_scores_selected_features = cross_val_score(knn_best, X_train_scaled, y_train, cv=5, scoring='accuracy')
mean_accuracy_selected = np.mean(cv_scores_selected_features)
std_accuracy_selected = np.std(cv_scores_selected_features)

y_pred = knn_best.predict(X_test_scaled)
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))
print(f"\nValidação cruzada com as melhores características (SelectKBest):\nAcurácia média: {mean_accuracy_selected:.4f}, Desvio padrão: {std_accuracy_selected:.4f}")

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.92      0.93      0.92       110
           1       0.93      0.92      0.92       109

    accuracy                           0.92       219
   macro avg       0.92      0.92      0.92       219
weighted avg       0.92      0.92      0.92       219

Matriz de Confusão:
 [[102   8]
 [  9 100]]

Validação cruzada com as melhores características (SelectKBest):
Acurácia média: 0.8978, Desvio padrão: 0.0193
